# Overlord AI
For information about this project, see [this README.md](README.md)


# CODE STARTS HERE

First we import all the libraries and whatnot

In [11]:
import pandas as pd
import glob

Definitions go here

In [12]:
dataPath = "../data/*.csv"
dfList = ["asdf"] # define a junk list
dfList.clear() # clear said junk list 


Read the data, using glob to get all the files at once

In [13]:
dataList = glob.glob(dataPath)
print(dataList)
for fileName in dataList:
    dfList.append(pd.read_csv(fileName , index_col=0))
    print(len(dfList))
    print(fileName)

['../data/ds_salaries copy.csv', '../data/ds_salaries.csv']
1
../data/ds_salaries copy.csv
2
../data/ds_salaries.csv


Method to show some info about the data

In [14]:
def showDataSummary(x):
    print("data type:") 
    print(type(df)) 
    print("10 samples:")
    print(x.sample(10)) # needs to be encapsulated in print()
    print("data describe:")
    print(x.describe()) # needs to be encapsulated in print()
    print("data info:")
    print(x.info())

Loop to run through all the dataframes in dfList

In [15]:
 for df in dfList:
    showDataSummary(df)
    print("AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA")
    #print(df)


data type:
<class 'pandas.core.frame.DataFrame'>
10 samples:
     AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA experience_level  \
229                                           2021               SE   
447                                           2022               SE   
549                                           2022               SE   
559                                           2022               SE   
39                                            2020               EN   
477                                           2022               SE   
171                                           2021               MI   
254                                           2021               MI   
187                                           2021               EX   
92                                            2021               MI   

    employment_type                  job_title   salary salary_currency  \
229              FT               Data Analyst    90000             CAD   
447    